# Week 19 - Deep Learning

မင်္ဂလာပါ။ Data Science Using Python - Week 19 က ကြိုဆိုပါတယ်။ 

ဒီအပါတ်မှာ ကျနော်တို့ **Deep Learning** ကို မိတ်ဆက်သဘောနဲ့ လေ့လာမှာ ဖြစ်ပါတယ်။ 

> Deep Learning ဟာ Machine Learning ရဲ့ အစိတ်အပိုင်းတခု၊ Machine Learning ဟာ Data Science ရဲ့ အစိတ်အပိုင်းတခုလို့ ဆိုပေမဲ့ Deep Learning နဲ့ ပတ်သက်တာအကုန်လုံးကို ဒီ course ထဲမှာ အကုန် ထည့်သွင်း သင်ကြားဖို့ လက်တွေ့မှာ မဖြစ်နိုင်ပါဘူး။

Deep Learning အကြောင်းကို မပြောခင် Machine Learning technique တခုရဲ့ သဘောကို ပြန်နွှေး လေ့လာကြည့်ရအောင်။

## Structure of a Machine Learning Application

In [ ]:
from sklearn import datasets as sk_ds
from sklearn import neural_network as sk_nn
from sklearn import preprocessing as sk_pp
from sklearn import model_selection as sk_ms
from sklearn import metrics as sk_metrics

import pandas as pd
import numpy as np

### 0. Data Preparation

In [ ]:
# 0. This is not ML; This is loading and preparing data
df_X, ds_y = sk_ds.fetch_openml("credit-g", as_frame=True, return_X_y=True)
df_X_tr, df_X_ts, ds_y_tr, ds_y_ts = sk_ms.train_test_split(df_X, ds_y, test_size=0.2, random_state=42, shuffle=True)

### 1. Feature Definition 

ဒီအဆင့်မှာ ဘယ် feature transformation (အလုပ်ထဲတွင် feature ဟုပဲ လုံးချပြီး ပြောတတ်) တွေကို ဘယ်လို အဆင့်ဆင့် သုံးမယ်ဆိုတာကို စဉ်းစား သတ်မှတ် ရပါတယ်။

> အရေးအကြီးဆုံးက test set ကို ဘယ်တော့မှ ထည့်ပြီး မ `fit` ဖို့ပါပဲ။

In [ ]:
# 1. Feature Definition
def fit_features(X: pd.DataFrame):
    X_num = X.select_dtypes(include=["number"])
    feature_1_ss = sk_pp.StandardScaler()
    feature_2_pf = sk_pp.PolynomialFeatures(degree=2)
    feature_1_ss.fit(X_num)
    X_f1 = feature_1_ss.transform(X_num)
    feature_2_pf.fit(X_f1)
    return (feature_1_ss, feature_2_pf)

feature_definitions = fit_features(df_X_tr)

### 2. Model Definition

ဒီအဆင့်မှာတော့ ဘယ် model definition (အလုပ်ထဲတွင် model ဟုပဲ လုံးချပြီး ပြောတတ်) ကို သုံးမယ်ဆိုတာကို စဉ်းစား သတ်မှတ်ရပါတယ်။ 

> course တခုလုံးမှာ init လို့ ပြောခဲ့တဲ့ အဆင့်ပါ။

1. ဒီလို စဉ်းစားရာမှာ linear model, svm, random forest, gradient boosting trees သို့မဟုတ် neural network စသဖြင့် စဉ်းစားရတာက ပထမအဆင့် ဖြစ်ပြီး
   
2. ဒုတိယအဆင့်အနေနဲ့ တခုချင်းစီရဲ့ hyper-parameter တွေကို သတ်မှတ်ပေးရပါတယ်။ 
   
   > ဥပမာအားဖြင့် `sklearn.linear_models.SGDClassifier` မှာဆိုရင် hyper-parameter အနေနဲ့ `l1_ratio`, `penalty` စတာတွေကို သတ်မှတ်ပေးရပါတယ်။ 
   အဲဒီကမှ model-parameter (`coef_` နဲ့ `intercept_` တို့) ကို learn ယူတာ ဖြစ်ပါတယ်။
   


In [ ]:
# 2. Model Definition - What model **architecture** to use
model_nn = sk_nn.MLPClassifier(
    # architecture of neural network
    hidden_layer_sizes=(20, 20), activation="relu", 
    # optimizer SGD, ADAM or whatever fancy one
    solver="adam", 
    # steps in train loop characteristics
    batch_size=100, learning_rate="constant", learning_rate_init=0.001, max_iter=10000, shuffle=True, random_state=42,
    # other model parameters
    alpha=0.0001)

### 3. Feature Computation

ဒီအဆင့်မှာ စောစောက သတ်မှတ်ထားတဲ့ feature definition တွေအတိုင်း feature value တွေ (အလုပ်ထဲမှာ feature လို့ပဲ လုံးချပြီး ပြော) ကို compute လုပ်ရတာ ဖြစ်ပါတယ်။ 

In [ ]:
# 3. Feature Computation
def transform_features(X: pd.DataFrame, features=()):
    X_num = X.select_dtypes(include=["number"])
    _features = X_num
    for f in features:
        _features = f.transform(_features)
    return _features

feature_values_tr = transform_features(df_X_tr, features=feature_definitions)

### 4. Model Training

ဒီအဆင့်မှာတော့ အဆင့် ၃ က ရထားတဲ့ feature values တွေကို အဆင့် ၂ မှာ သတ်မှတ်ထားတဲ့ model definition နဲ့ train (`fit`) တာ ဖြစ်ပါတယ်။

In [ ]:
# 4. Model Training

def fit_model(model, X, y):
    model.fit(X, y)
    return model

model_nn = fit_model(model_nn, feature_values_tr, ds_y_tr)

### 5. Model Validation

ဒုတိယ နောက်ဆုံးအဆင့်ဖြစ်တဲ့ ဒီအဆင့်မှာတော့ ရလာတဲ့ trained model ကို စာမေးပွဲစစ်သလို unseen data နဲ့ စစ်ဆေးတဲ့ သဘောဖြစ်ပါတယ်။

In [ ]:
# 5. Model Validation

def validate_model(features, model, X_ts, y_ts):
    feature_values_ts = transform_features(X_ts, features=features)
    y_hat = model.predict(feature_values_ts)
    print (sk_metrics.classification_report(y_ts, y_hat, sample_weight=[{"good": 1, "bad": 5}[_y] for _y in y_ts]))
    return feature_values_ts

feature_values_ts = validate_model(features=feature_definitions, model=model_nn, X_ts=df_X_ts, y_ts=ds_y_ts)

### 6. Model Application

Model Scoring/Inferencing လို့လဲ ခေါ်ကြတဲ့ ဒီ နောက်ဆုံး အဆင့်ကတော့ validation လုပ်လို့ ကျေနပ်မှ (model စာမေးပွဲအောင်မှ) ဆက်လုပ်ကြတာပါ။

> Validation မှာ performance မကောင်းရင် ဘာဆက်လုပ်ကြမလဲ ??? 

စောစောက feature definition တွေနဲ့ model definition တွေကို production ကို ယူသွားပြီး data အသစ်တွေကို apply လုပ်ကြပါတယ်။ 

```python
# how to move features and models to production ?

# 1. pickle them in training machine

import pickle
with open("feats_and_model.bin", "wb") as f:
    pickle.dump({
        "features": feature_definitions, 
        "model": model_nn
    }, f)

# 2. un-pickle them in production machine

import pickle
with open("feats_and_model.bin", "rb") as f:
    _fm = pickle.load(f)
    feature_definitions = _fm["features"]
    model_nn = _fm["model"]

# use them with new data 
```

### Summary

အကျဉ်းချုပ် အနေနဲ့ Machine Learning Application တခုရဲ့ structure ဟာ ဒီလို လာပါတယ်။

```python
df_X_tr, df_X_ts, ds_y_tr, ds_y_ts = load_and_split_data()

feature_def = get_feature_definition()
feature_def.fit(df_X_tr)

model_def = get_model_definition(
    model_architecture,
    model_optimizer,
    model_step
)

feature_values_tr = feature_def.transform(df_X_tr)
model_def.fit(feature_values_tr, ds_y_tr)

feature_values_ts = feature_def.transform(df_X_ts)
y_hat = model_def.predict(feature_values_ts)
print (classification_report(ds_y_ts, y_hat))
```

> အရေးပါတဲ့ မှတ်သားစရာတွေက 
> * `model_def` မှာ `model_architecture`, `model_optimizer` နဲ့ `model_step` ဆိုပြီး ၃ ပိုင်းပါပါတယ်။
> * trained model ရဲ့ performance ပေါ်မူတည်ပြီး `feature_def` ကို သွားသွားပြန်ပြောင်းပြီး စမ်းဖို့ လိုပါတယ်။ 

## Deep Learning


Deep Learning ဟာ hidden layer 1 ခုထက် ပိုပါတဲ့ Neural Network ပဲ ဖြစ်တယ်။ 

Computing efficiency ကြောင့်ပဲလို့ ဆိုဆို၊ ဘာကြောင့်ပဲလို့ ဆိုဆို width ရော၊ breadth ရော၊ depth ရော ကြီးတဲ့ neural network model တွေအနေနဲ့ `sklearn.neural_network` ထဲက model တွေလိုမျိုး **fully connected** ဖြစ်လို့ အဆင်မပြေပါဘူး။ 

ဒါကြောင့် neural network ရဲ့ definition (အလုပ်ထဲတွင် architecture ဟု ပြော) ကို စိတ်ကြိုက် ပြောင်းပေးနိုင်ဖို့ လိုလာပါတယ်။ 

အဲဒါကို လုပ်ပေးနိုင်တဲ့ library ၂ ခု ရှိပါတယ်။ 

* tensorflow by Google and
* torch by Facebook 

တို့ ဖြစ်ပါတယ်။ 

tensorflow က powerful ပိုဖြစ်ပြီး torch (pytorch) က လေ့လာရတာ ပိုလွယ်ပါတယ်။ ဒါကြောင့် ဒီနေ့ `torch` ကို စမ်းကြည့်ကြပါမယ်။ 

In [ ]:
import torch
import torch.nn.functional as F

အောက်က model definition နဲ့ တူတဲ့ model တခုကို ဆောက်ကြည့်ရအောင်။

```python
# 2. Model Definition - What model **architecture** to use
model_nn = sk_nn.MLPClassifier(
    # architecture of neural network
    hidden_layer_sizes=(20, 20), activation="relu", 
    # optimizer SGD, ADAM or whatever fancy one
    solver="adam", 
    # steps in train loop characteristics
    batch_size=100, learning_rate="constant", learning_rate_init=0.001, max_iter=10000, shuffle=True, random_state=42,
    # other model parameters
    alpha=0.0001)
```

In [ ]:
input_layer_size = feature_definitions[-1].n_output_features_ # the number of columns in X
hidden_layer_sizes = (20, 20)
output_layer_size = 2 # it has good and bad (2 classes)

# architecture of neural network (including activation)
class TorchModel(torch.nn.Module):

    def __init__(self):
        super(TorchModel, self).__init__()

        self.layer1 = torch.nn.Linear(input_layer_size, hidden_layer_sizes[0])
        self.relu1 = torch.nn.ReLU()
        self.layer2 = torch.nn.Linear(hidden_layer_sizes[0], hidden_layer_sizes[1])
        self.relu2 = torch.nn.ReLU()
        self.output = torch.nn.Linear(hidden_layer_sizes[1], output_layer_size)
        self.relu3 = torch.nn.ReLU()
        
    def forward(self, x):
        x_1 = self.relu1(self.layer1(x))
        x_2 = self.relu2(self.layer2(x_1))
        x_3 = self.relu3(self.output(x_2))
        return x_3

model_t = TorchModel()

In [ ]:
# optimizer
learning_rate_init=0.001
solver = torch.optim.Adam(model_t.parameters(), lr=0.001)

In [ ]:
# data loading
from torch.utils.data import TensorDataset, DataLoader

batch_size = 100

X_tr_tensor = torch.Tensor(feature_values_tr)
X_ts_tensor = torch.Tensor(feature_values_ts)
y_tr_tensor = torch.Tensor([{"good": 0, "bad": 1}[_y] for _y in ds_y_tr]).long()
y_ts_tensor = torch.Tensor([{"good": 0, "bad": 1}[_y] for _y in ds_y_ts]).long()

tr_dataset = TensorDataset(X_tr_tensor, y_tr_tensor)

train_loader = DataLoader(tr_dataset, batch_size=batch_size)


In [ ]:
# training loop -- including model train step
max_iter=10000

loss_func = torch.nn.CrossEntropyLoss()

for epoch in range(max_iter):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero-set the parameter gradients; important
        solver.zero_grad()

        # forward + backward + optimize
        outputs = model_t(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        solver.step()

        running_loss += loss.item()
    if epoch % 2000 == 1999:    # print every 2000 epoch
        print('epoch = {epoch} : loss = {running_loss}'.format(epoch=epoch, running_loss=running_loss/2000.0))
        running_loss = 0.0

print('Finished Training')

In [ ]:
model_t.eval()
with torch.no_grad():
    y_hat = model_t(X_ts_tensor)
    y_predicted = [0 if y_[0] > y_[1] else 1 for y_ in F.softmax(y_hat).long()]
    print (sk_metrics.classification_report(y_ts_tensor.detach(), y_predicted, sample_weight=[{"good":1, "bad": 5}[_y] for _y in ds_y_ts]))

### Deep Analysis of Deep Learning

`sklearn` package များနှင့် deep learning (`torch`) တို့၏ ကွဲပြားခြားနားချက်များမှာ 

* model architecture, အတွင်းပိုင်း ချိတ်ဆက်ပုံနှင့် အဖြေထုတ်ပုံတို့ကို deep learning တွင် user ကိုယ်တိုင် သတ်မှတ်ရခြင်း ဖြစ်သည်။ `sklearn` တွင်မူ model များ၏ အလုပ်လုပ်ပုံမှာ အသေဖြစ်ပြီး hyper parameter များမှတဆင့် အနည်းငယ်သာ ပြောင်းလဲ၍ ရသည်။

* Feature Engineering ပိုင်းအတွက် မည်သည့် Feature Function ကို သုံးမည်၊ မည်သို့ ချိတ်ဆက်မည်ကို deep learning တွင် အလိုအလျောက် learn နိုင်သည်။ layer များများ ထည့်ပြီး compute resources များများ သုံးရန်သာ လိုသည်။

* ပိုမို မြန်ဆန်အောင် GPU သုံး၍ ရသည်။ 

> အခု ကျနော်တို့ GPU သုံးတာ လက်တွေ့ ပြမှာမို့လို့ Runtime ကို restart လုပ်ပေးပါ။

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from sklearn import datasets as sk_ds 
from sklearn import model_selection as sk_ms
from sklearn import preprocessing as sk_pp
from sklearn import metrics as sk_metrics

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
df_X, ds_y = sk_ds.fetch_openml("credit-g", as_frame=True, return_X_y=True)

df_X_tr, df_X_ts, ds_y_tr, ds_y_ts = sk_ms.train_test_split(df_X, ds_y, test_size=0.2, random_state=42, shuffle=True)

ohe = sk_pp.OneHotEncoder(sparse=False, handle_unknown="ignore")
ohe.fit(df_X_tr.select_dtypes(include=["category"]))
X_tr = pd.DataFrame(data=ohe.transform(df_X_tr.select_dtypes(include=["category"])), index=df_X_tr.index, columns=ohe.get_feature_names_out())
X_ts = pd.DataFrame(data=ohe.transform(df_X_ts[ohe.feature_names_in_]), index=df_X_ts.index, columns=ohe.get_feature_names_out())

X_tr = pd.concat((X_tr, df_X_tr.select_dtypes(include=["number"])), axis="columns")
X_ts = pd.concat((X_ts, df_X_ts.select_dtypes(include=["number"])), axis="columns")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
input_layer_size = X_tr.shape[1]
output_layer_size = 2
class TorchModel0(torch.nn.Module):

    def __init__(self):
        super(TorchModel0, self).__init__()

        self.layers = torch.nn.Sequential(
            torch.nn.Linear(input_layer_size, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 2),
            torch.nn.ReLU()
        )
        
    def forward(self, x):
        return self.layers(x)

model_t0 = TorchModel0()

In [ ]:
# optimizer
learning_rate_init=0.001
solver0 = torch.optim.SGD(params=model_t0.parameters(), lr=learning_rate_init, momentum=0.9)

In [ ]:
# data loading
from torch.utils.data import TensorDataset, DataLoader

batch_size = 100

X_tr_tensor = torch.Tensor(X_tr.values).to(device)
X_ts_tensor = torch.Tensor(X_ts.values).to(device)
y_tr_tensor = torch.Tensor([{"good": 0, "bad": 1}[_y] for _y in ds_y_tr]).long().to(device)
y_ts_tensor = torch.Tensor([{"good": 0, "bad": 1}[_y] for _y in ds_y_ts]).long().to(device)

tr_dataset = TensorDataset(X_tr_tensor, y_tr_tensor)

train_loader = DataLoader(tr_dataset, batch_size=batch_size)

In [ ]:
# training loop -- including model train step
def train_model(max_iter=10000, model_t=None, solver=None, loss_func=None):

    model_t = model_t.to(device)

    for epoch in range(max_iter):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # zero-set the parameter gradients; important
            solver.zero_grad()

            # forward + backward + optimize
            outputs = model_t(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            solver.step()

            running_loss += loss.item()
        if epoch % 2000 == 1999:    # print every 2000 epoch
            print('epoch = {epoch} : loss = {running_loss}'.format(epoch=epoch, running_loss=running_loss/2000.0))
            running_loss = 0.0

    print('Finished Training')

    return model_t

In [ ]:
model_t0 = train_model(model_t=model_t0, solver=solver0, loss_func = torch.nn.CrossEntropyLoss())

In [ ]:
model_t0.eval()
with torch.no_grad():
    y_hat = model_t0(X_ts_tensor)
    y_predicted = [0 if y_[0] > y_[1] else 1 for y_ in F.softmax(y_hat).long()]
    print (sk_metrics.classification_report(y_ts_tensor.detach().cpu(), y_predicted, sample_weight=[{"good":1, "bad": 5}[_y] for _y in ds_y_ts]))

အောက်က model နောက်တခုမှာတော့ layer တွေတိုးပြီး feature engineering လုပ်သလိုပဲ performance တိုးအောင် လုပ်တာကို စမ်းသပ်မှာ ဖြစ်ပါတယ်။

In [ ]:
# architecture of neural network (including activation)
input_layer_size = X_tr.shape[1]
output_layer_size = 2
class TorchModel(torch.nn.Module):

    def __init__(self):
        super(TorchModel, self).__init__()

        # ဒီနေရာမှာ အဆင့်တွေ ပိုများသွားတာ သတိပြုပါ
        self.layers = torch.nn.Sequential(
            torch.nn.BatchNorm1d(input_layer_size),
            torch.nn.Linear(input_layer_size, 100),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(100),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(100, 50),
            torch.nn.ReLU(),
            torch.nn.Linear(50, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, output_layer_size)
        )
        
    def forward(self, x):
        return self.layers(x)

model_t = TorchModel()

In [ ]:
# optimizer
learning_rate_init=0.001
solver = torch.optim.SGD(params=model_t.parameters(), lr=learning_rate_init, momentum=0.9)

In [ ]:
model_t = train_model(model_t=model_t, solver=solver, loss_func = torch.nn.CrossEntropyLoss())

In [ ]:
model_t.eval()
with torch.no_grad():
    y_hat = model_t(X_ts_tensor)
    y_predicted = [0 if y_[0] > y_[1] else 1 for y_ in F.softmax(y_hat).long()]
    print (sk_metrics.classification_report(y_ts_tensor.detach().cpu(), y_predicted, sample_weight=[{"good":1, "bad": 5}[_y] for _y in ds_y_ts]))

### About Cross Entropy Loss

Cross Entropy Loss ဆိုတာကြီးဟာ ကြောက်စရာကောင်းတဲ့ သင်္ချာ ဖော်မြူလာကြီးလို ဖြစ်နေပါတယ်။ 

ဒါကို ကြောက်စရာ မကောင်းတော့အောင်လို့ ၂ ပိုင်းခွဲပြီး ပြောပါမယ်။ 

* ပထမပိုင်းက Entropy ပါ
* ဒုတိယပိုင်းက Negative Log Likelihood Loss ပါ။

#### Entropy

[Reference](https://www.youtube.com/watch?v=YtebGVx-Fxw&t=1s)

**surprise**

ပထမပိုင်းရဲ့ ပထမဆုံး အနေနဲ့ surprise ကို ပြောကြရအောင်။

> ဘောလုံးအသင်းတသင်းက အရမ်းတော်တယ်ဆိုပါစို့။ ဒါဆို ကစားတဲ့ ပွဲတပွဲမှာ သူနိုင်ဖို့က များမှာ။ ဆိုပါစို့ $P(\mathrm{win})=0.9$ နဲ့ $P(\mathrm{lose})=0.1$
> ဒါဆို surprise ဆိုတာ သူနိုင်လာရင် သိပ် surprise မဖြစ်ဘူး။ သူရှုံးလာရင်တော့ surprise က များတယ်။ 

ဒါကြောင့် surprise $s(\mathrm{win})$ က နည်းပြီး $s(\mathrm{lose})$ က များရမယ်။ 

ဒါကြောင့် ဒီလို definition သတ်မှတ်မယ် လို့ စဉ်းစားကြည့်ရအောင်။ 

$s(X) = \frac{1}{P(X)}$

> $s(\mathrm{win}) = 1/0.9 = 1.111$ နည်းတယ်
> $s(\mathrm{lose}) = 1/0.1 = 10.000$ များတယ်

ဒီအထိတော့ ဟုတ်နေတာပဲ။ 

ခက်တာက အရမ်း အရမ်း x တသိန်း တော်တဲ့ အသင်း ($P(\mathrm{win}) = 1.0$)ကျတော့ နိုင်ရင် ဖြစ်မဲ့ surprise ဆိုတာ သုည (ဇီးရိုး) ဖြစ်သင့်တယ်။ 

> $s(\mathrm{win}) = 1/1.0 = 1$ မဖြစ်ဘူး။

ဒီတော့ သင်္ချာ ပညာရှင်တွေ ပီပီ အားနေရင် $log$ ယူလိုက်တယ်။ 

surprise $s(X) = log(\frac{1}{P(X)})$

> $log(1) = 0$ ဖြစ်တာမို့ အဆင်ပြေသွားတယ်။

**expected value**

ဒုတိယ ဆက်ပြောရမှာကတော့ expected value ပါ။ 

Expected Value 
$$E = \sum X.P(X)$$

လို့ အဓိပ္ပါယ် သတ်မှတ်ချက် ရှိတယ်။

> ဥပမာ weighted အံစာတုံး တတုံးမှာ $P(X=1) = 0.5, P(X\neq 1) = 0.1$ ဆိုပါစို့။
> 
> သူ့ရဲ့ Expected Value ကို number of trail (ပစ်တဲ့ အကြိမ်ရေ) နဲ့ မြှောက်ရင် အံစာတုံးပေါ်က ဂဏန်းတွေ ပစ်ပြီး ပေါင်းထားတာနဲ့ နီးစပ်လိမ့်မယ်။ 

In [20]:
faces = [1, 2, 3, 4, 5, 6]
probabilites = np.array([0.5] + [0.1 for _ in range(2, 6+1)])

for f, p in zip(faces, probabilites):
    print ("probability of {} : {}".format(f, p))

expected_value = np.array([f * p for f, p in zip(faces, probabilites)]).sum()
print ("expected_value : {}".format(expected_value))

number_of_trail = 1000
print ("expected_value x number_of_trial : {}".format(expected_value * number_of_trail))

actual_trials = np.random.choice(faces, replace=True, p=probabilites, size=number_of_trail)
sum_of_actual_trials = actual_trials.sum()

print ("sum of actual trials : {}".format(sum_of_actual_trials))

probability of 1 : 0.5
probability of 2 : 0.1
probability of 3 : 0.1
probability of 4 : 0.1
probability of 5 : 0.1
probability of 6 : 0.1
expected_value : 2.5
expected_value x number_of_trial : 2500.0
sum of actual trials : 2560


**entropy**

ဒီတော့ entropy ဆိုတာကို surprise $s(X)$ တွေရဲ့ expected value အဖြစ် သတ်မှတ်လိုက်ရအောင်။ 

Entropy $$S = \sum s(X).P(X) = \sum P(X).s(X)$$

ဒီတော့ အပေါ်က $s(X)$ ရဲ့ ဖော်မြူလာ အစားသွင်းတော့ 

$$S = \sum P(X)log(\frac{1}{P(X)})$$

log ရဲ့ definition အရ $log(\frac{a}{b}) = log(a) - log(b)$ ဖြစ်တဲ့အတွက်

$$S = \sum \left[ P(X) ( log(1) - log(P(X)) ) \right]$$

$P(X)$ က အထဲ ဝင်မြှောက်တော့

$$S = \sum \left[ P(X).log(1) - P(X).log(P(X))\right]$$

$log(1) = 0$ ဖြစ်တာမို့

$$S = \sum \left[- P(X).log(P(X))\right]$$

အထဲက အနှုတ်ကို အပြင်ထုတ်လိုက်တော့ 

$$S = - \sum P(X).log(P(X))$$

ဆိုပြီး entropy ရဲ့ formula ရလာပါတယ်။ 

ဒုတိယပိုင်းကို စပါမယ်။

**likelihood**

ဒုတိယပိုင်းရဲ့ စစချင်းမှာ likelihood $\mathcal{L}$ ဆိုတာကို define ပါမယ်။ 

> model တခုရဲ့ likelihood ဆိုတာ ဒီ model ရဲ့ မှန်နိုင်ခြေလို့ အရပ်ပြော ပြောလို့ရပါတယ်။
>
> model က classes $[0, 1, \dots, C]$ တွေအတွက် ထုတ်ပေးတဲ့ probabilities $[p_0, p_1, \dots, p_C]$ တွေထဲက အကြီးဆုံး $p_{c\_max}$ ရဲ့ class $c\_max$ ဟာ $y$ တွေနဲ့ တူနေရင် likelihood များပြီး မတူရင် likelihood နည်းပါမယ်။ 

ဒီတော့ **sample တခု (ONE sample)** အတွက် likelihood $\mathcal{L}_i$ ကို စဉ်းစားကြည့်ရအောင်။

$$\mathcal{L}_i(f|\langle X_i \in X, y_i \in y \rangle) = P(\langle X_i \in X, y_i \in y \rangle|f) = f(x_i)_{y_i}$$ 

model $f$ က ထုတ်ပေးတဲ့ probability $f(x_i) = $[p_0, p_1, \dots, p_C]$ တွေထဲက $p_{y_i}$ ဟာ ကြီးလေလေ likelihood က များလေလေပါပဲ။

$X$ ထဲမှာ ရှိကြတဲ့ sample (row) တွေဟာ တခုနဲ့ တခု independent ဖြစ်ကြတဲ့အတွက် Product Rule အရ

$$\mathcal{L}(f|X, y) = P(X, y|f) = \prod_{x_i \in X, y_i \in y} f(x_i)_{y_i}$$

**maximum likelihood**

အကောင်းဆုံး model ဆိုတာဟာ အပေါ်က အီကွေးရှင်းကို များနိုင်သမျှများအောင် maximize ဖြစ်စေတဲ့ model $f$ ပါပဲ။ 

Best model $\theta$ is

$$\theta = \argmax_{f \in F} \mathcal{L}(f|X, y) = \argmax_{f \in F} \prod f(X_i)_{y_i}$$

**maximum log likelihood** 

ဒီမှာလဲ $log(ab) = log(a) + log(b)$ ဖြစ်တဲ့အတွက် အပေါ်ကဟာကို maximize တာဟာ အောက်ကဟာကို maximize တာနဲ့ တူတာမို့

$$\theta = \argmax_{f \in F} \mathcal{L}(f|X, y) = \argmax_{f \in F} \sum \left[ log(f(X_i)_{y_i}) \right]$$

**minimize negative**

Maximize မလုပ်ချင်ရင် -1 နဲ့ မြှောက်ပြီး Minimize လို့ ရတာမို့ ... 

အကောင်းဆုံး model $\theta$ ဟာ 

$$\theta = \argmin_{f \in F} - \mathcal{L}(f|X, y) = \argmin_{f \in F} - \sum \left[ log(f(X_i)_{y_i}) \right]$$

> တခု သတိထားဖို့က for any $i$, $P(y_i) = 1$

ဒါကြောင့် ဒီလို ပြင်ရေးမယ်။ 

$$\theta = \argmin_{f \in F} - \mathcal{L}(f|X, y) = \argmin_{f \in F} - \sum \left[ P(y_i).log(f(X_i)_{y_i}) \right]$$

**Entropy နှင့် သွားတူနေမှု**

နောက်တခု သတိရဖို့ ... 

> $f(X_i)_{y_i}$ က ဘာလဲ 

စကတည်းက define ထားတာ probability လေ။ $Q(y_i) = f(X_i)_{y_i}$ လို့ ထားလိုက်မယ်ဆိုရင်

ဒီတော့ entropy formula $S = - \sum P(X).log(P(X))$ နဲ့ $- \sum \left[ P(y_i).log(Q(y_i))\right]$ ဆင်ဆင် တူ မနေဘူးလား။

တကယ်က $- \sum \left[ P(y_i).log(Q(y_i))\right]$ ဆိုတာ cross entropy ရဲ့ formula ပဲ။ 